In [5]:
import os

folder = 'images'
files = os.listdir(folder)

# Optionally, sort the files if you want a specific order.
files.sort()
ext = ".jpg"

for i, filename in enumerate(files):

    # Construct full path to the current file
    src = os.path.join(folder, filename)
    # If you want to preserve the file extension:
    # ext = os.path.splitext(filename)[1]
    # Construct the new file name (e.g., 0.jpg, 1.jpg, etc.)
    new_name = os.path.join(folder, f"{i}{ext}")
    os.rename(src, new_name)
    print(f"Renamed '{filename}' to '{i}{ext}'")


Renamed '1000.jpg' to '0.jpg'
Renamed '1001.jpg' to '1.jpg'
Renamed '1002.jpg' to '2.jpg'
Renamed '1003.jpg' to '3.jpg'
Renamed '1004.jpg' to '4.jpg'
Renamed '1005.jpg' to '5.jpg'
Renamed '1006.jpg' to '6.jpg'
Renamed '1007.jpg' to '7.jpg'
Renamed '1008.jpg' to '8.jpg'
Renamed '1009.jpg' to '9.jpg'
Renamed '1010.jpg' to '10.jpg'
Renamed '1011.jpg' to '11.jpg'
Renamed '1012.jpg' to '12.jpg'
Renamed '1013.jpg' to '13.jpg'
Renamed '1014.jpg' to '14.jpg'
Renamed '1015.jpg' to '15.jpg'
Renamed '1016.jpg' to '16.jpg'
Renamed '1017.jpg' to '17.jpg'
Renamed '1018.jpg' to '18.jpg'
Renamed '1019.jpg' to '19.jpg'
Renamed '1020.jpg' to '20.jpg'
Renamed '1021.jpg' to '21.jpg'
Renamed '1022.jpg' to '22.jpg'
Renamed '1023.jpg' to '23.jpg'
Renamed '1024.jpg' to '24.jpg'
Renamed '1025.jpg' to '25.jpg'
Renamed '1026.jpg' to '26.jpg'
Renamed '1027.jpg' to '27.jpg'
Renamed '1028.jpg' to '28.jpg'
Renamed '1029.jpg' to '29.jpg'
Renamed '1030.jpg' to '30.jpg'
Renamed '1031.jpg' to '31.jpg'
Renamed '1032.jpg'

In [ ]:
# %% [code]
import os
import json
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient

# Replace with your actual Azure endpoint and API key
endpoint = ""
key = ""

# Initialize the Document Intelligence client
client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))

input_folder = "images"           # Folder with input images
output_folder = "images_json"        # Folder to save JSON outputs
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.lower().endswith((".png", ".jpg", ".jpeg", ".tiff", ".bmp")):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_path} ...")
        with open(file_path, "rb") as file_stream:
            poller = client.begin_analyze_document("prebuilt-receipt", 
                                                     body=file_stream, 
                                                     content_type="application/octet-stream")
            result = poller.result()
        # Save JSON output to file
        json_file_name = os.path.splitext(file_name)[0] + ".json"
        json_file_path = os.path.join(output_folder, json_file_name)
        with open(json_file_path, "w", encoding="utf-8") as f:
            json.dump(result.as_dict(), f, indent=4)
        print(f"Saved result to {json_file_path}")


Processing images\0.jpg ...
Saved result to images_json\0.json
Processing images\1.jpg ...
Saved result to images_json\1.json
Processing images\10.jpg ...
Saved result to images_json\10.json
Processing images\100.jpg ...
Saved result to images_json\100.json
Processing images\101.jpg ...
Saved result to images_json\101.json
Processing images\102.jpg ...
Saved result to images_json\102.json
Processing images\103.jpg ...
Saved result to images_json\103.json
Processing images\104.jpg ...
Saved result to images_json\104.json
Processing images\105.jpg ...
Saved result to images_json\105.json
Processing images\106.jpg ...
Saved result to images_json\106.json
Processing images\107.jpg ...
Saved result to images_json\107.json
Processing images\108.jpg ...
Saved result to images_json\108.json
Processing images\109.jpg ...
Saved result to images_json\109.json
Processing images\11.jpg ...
Saved result to images_json\11.json
Processing images\110.jpg ...
Saved result to images_json\110.json
Process

In [6]:
import json 
import os

def polygon_to_yolo(polygon, image_width, image_height):
    """
    Convert a polygon (list of 8 numbers: [x1, y1, x2, y2, x3, y3, x4, y4])
    to YOLO format (x_center, y_center, width, height) normalized by image dimensions.
    """
    xs = polygon[0::2]
    ys = polygon[1::2]
    xmin = min(xs)
    xmax = max(xs)
    ymin = min(ys)
    ymax = max(ys)
    x_center = (xmin + xmax) / 2.0 / image_width
    y_center = (ymin + ymax) / 2.0 / image_height
    box_width = (xmax - xmin) / image_width
    box_height = (ymax - ymin) / image_height
    return x_center, y_center, box_width, box_height

def extract_seller_names(fields, image_width, image_height, max_entities=2):
    """
    Extract seller name from any of these fields:
      SellerName, MerchantName, BranchName, ShopName, OutletName.
    Output as YOLO class 0.
    """
    seller_keys = {"SellerName", "MerchantName", "BranchName", "ShopName", "OutletName"}
    annotations = []
    for key in seller_keys:
        field = fields.get(key, {})
        if field.get("boundingRegions"):
            polygon = field["boundingRegions"][0].get("polygon")
            if polygon:
                bbox = polygon_to_yolo(polygon, image_width, image_height)
                annotations.append(f"0 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
    return annotations[:max_entities]

def process_json_file(json_file, output_folder):
    """
    Process a single JSON file and extract fields with the following YOLO mapping:
      0: Seller Name
      1: Seller VAT Number
      2: Document Date
      3: Product Description (per item)
      4: Quantity (per item)
      5: Price (per item; use 'Price' if available, otherwise 'TotalPrice')
      6: Total Due Amount
    """
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error reading {json_file}: {e}")
        return

    if "pages" not in data or "documents" not in data:
        print(f"Missing required keys in {json_file}. Skipping.")
        return

    page = data["pages"][0]
    image_width = page.get("width", 640)
    image_height = page.get("height", 640)
    yolo_annotations = []

    document = data["documents"][0]
    fields = document.get("fields", {})

    # --- Class 0: Seller Name ---
    seller_annotations = extract_seller_names(fields, image_width, image_height, max_entities=2)
    if seller_annotations:
        yolo_annotations.extend(seller_annotations)
    else:
        print(f"Warning: No seller name field found in {json_file}.")

    # --- Class 1: Seller VAT Number ---
    seller_vat = fields.get("SellerVATNumber") or fields.get("seller_vat_number")
    if seller_vat and seller_vat.get("boundingRegions"):
        polygon = seller_vat["boundingRegions"][0].get("polygon")
        if polygon:
            bbox = polygon_to_yolo(polygon, image_width, image_height)
            yolo_annotations.append(f"1 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
    else:
        # Fallback: scan OCR lines for VAT information
        vat_found = False
        for line in page.get("lines", []):
            content = line.get("content", "")
            if "Vat Code" in content:
                polygon = line.get("polygon")
                if polygon:
                    bbox = polygon_to_yolo(polygon, image_width, image_height)
                    yolo_annotations.append(f"1 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
                    vat_found = True
                    break
        if not vat_found:
            print(f"Warning: Seller VAT Number field not found in {json_file}.")

    # --- Class 2: Document Date ---
    doc_date = fields.get("TransactionDate") or fields.get("Doc_date")
    if doc_date and doc_date.get("boundingRegions"):
        polygon = doc_date["boundingRegions"][0].get("polygon")
        if polygon:
            bbox = polygon_to_yolo(polygon, image_width, image_height)
            yolo_annotations.append(f"2 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
    else:
        print(f"Warning: Document Date field not found in {json_file}.")

    # --- Process Items for Table Details ---
    items_field = fields.get("Items", {})
    if items_field.get("valueArray"):
        for item in items_field["valueArray"]:
            value_object = item.get("valueObject", {})

            # Class 3: Product Description
            description = value_object.get("Description", {})
            if description.get("boundingRegions"):
                polygon = description["boundingRegions"][0].get("polygon")
                if polygon:
                    bbox = polygon_to_yolo(polygon, image_width, image_height)
                    yolo_annotations.append(f"3 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")

            # Class 4: Quantity (quantity_item)
            quantity = value_object.get("Quantity", {})
            if quantity.get("boundingRegions"):
                polygon = quantity["boundingRegions"][0].get("polygon")
                if polygon:
                    bbox = polygon_to_yolo(polygon, image_width, image_height)
                    yolo_annotations.append(f"4 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")

            # Class 5: Price (price)
            # Use the "Price" field if available; otherwise fallback to "TotalPrice"
            price = value_object.get("Price", {}) or value_object.get("TotalPrice", {})
            if price.get("boundingRegions"):
                polygon = price["boundingRegions"][0].get("polygon")
                if polygon:
                    bbox = polygon_to_yolo(polygon, image_width, image_height)
                    yolo_annotations.append(f"5 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
    else:
        print(f"Warning: No Items found in {json_file} under 'Items'.")

    # --- Class 6: Total Due Amount ---
    total_field = fields.get("Total") or fields.get("total_due_amount")
    if total_field and total_field.get("boundingRegions"):
        polygon = total_field["boundingRegions"][0].get("polygon")
        if polygon:
            bbox = polygon_to_yolo(polygon, image_width, image_height)
            yolo_annotations.append(f"6 {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}")
    else:
        print(f"Warning: Total Due Amount field not found in {json_file}.")

    # --- Write YOLO annotations to output file ---
    base_name = os.path.splitext(os.path.basename(json_file))[0]
    output_file = os.path.join(output_folder, base_name + ".txt")
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for annotation in yolo_annotations:
                f.write(annotation + "\n")
        print(f"Saved YOLO annotations to {output_file}")
    except Exception as e:
        print(f"Error writing {output_file}: {e}")

def process_json_folder(json_folder, output_folder):
    """
    Process all JSON files in the given folder and generate YOLO annotation files.
    """
    os.makedirs(output_folder, exist_ok=True)
    json_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.lower().endswith(".json")]
    if not json_files:
        print("No JSON files found in the specified folder.")
        return

    for json_file in json_files:
        process_json_file(json_file, output_folder)

# === Example usage ===

json_folder = "images_json"       # Folder where your JSON files are stored
yolo_output_folder = "yolo_annotations"  # Folder to save the YOLO-format annotation files

process_json_folder(json_folder, yolo_output_folder)
print("Processing complete!")


Saved YOLO annotations to yolo_annotations\0.txt
Saved YOLO annotations to yolo_annotations\1.txt
Saved YOLO annotations to yolo_annotations\10.txt
Saved YOLO annotations to yolo_annotations\100.txt
Saved YOLO annotations to yolo_annotations\101.txt
Saved YOLO annotations to yolo_annotations\102.txt
Saved YOLO annotations to yolo_annotations\103.txt
Saved YOLO annotations to yolo_annotations\104.txt
Saved YOLO annotations to yolo_annotations\105.txt
Saved YOLO annotations to yolo_annotations\106.txt
Saved YOLO annotations to yolo_annotations\107.txt
Saved YOLO annotations to yolo_annotations\108.txt
Saved YOLO annotations to yolo_annotations\109.txt
Saved YOLO annotations to yolo_annotations\11.txt
Saved YOLO annotations to yolo_annotations\110.txt
Saved YOLO annotations to yolo_annotations\111.txt
Saved YOLO annotations to yolo_annotations\112.txt
Saved YOLO annotations to yolo_annotations\113.txt
Saved YOLO annotations to yolo_annotations\114.txt
Saved YOLO annotations to yolo_annota

In [7]:
# %% [code]
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths to your original images and YOLO annotations
original_images_folder = "images"
yolo_annotations_folder = "yolo_annotations"

# Create the YOLO dataset folder structure
dataset_dir = "yolo_dataset"
images_train_dir = os.path.join(dataset_dir, "images", "train")
images_val_dir   = os.path.join(dataset_dir, "images", "val")
labels_train_dir = os.path.join(dataset_dir, "labels", "train")
labels_val_dir   = os.path.join(dataset_dir, "labels", "val")

for folder in [images_train_dir, images_val_dir, labels_train_dir, labels_val_dir]:
    os.makedirs(folder, exist_ok=True)

# List all image files (adjust extensions as needed)
all_images = sorted([f for f in os.listdir(original_images_folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))])

# Split into train (80%) and val (20%)
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

# Copy training images and labels
for img in train_imgs:
    shutil.copy(os.path.join(original_images_folder, img), os.path.join(images_train_dir, img))
    txt_name = os.path.splitext(img)[0] + ".txt"
    src_txt = os.path.join(yolo_annotations_folder, txt_name)
    dst_txt = os.path.join(labels_train_dir, txt_name)
    if os.path.exists(src_txt):
        shutil.copy(src_txt, dst_txt)

# Copy validation images and labels
for img in val_imgs:
    shutil.copy(os.path.join(original_images_folder, img), os.path.join(images_val_dir, img))
    txt_name = os.path.splitext(img)[0] + ".txt"
    src_txt = os.path.join(yolo_annotations_folder, txt_name)
    dst_txt = os.path.join(labels_val_dir, txt_name)
    if os.path.exists(src_txt):
        shutil.copy(src_txt, dst_txt)

print("Dataset folder structure created!")


Dataset folder structure created!


In [8]:
data_yaml_content = f"""
train: {os.path.abspath(images_train_dir)}
val: {os.path.abspath(images_val_dir)}
nc: 7
names: ['SellerName', 'SellerVAT', 'DocumentDate', 'ProductDescription', 'Quantity', 'Price', 'TotalDue']
"""

yaml_file_path = os.path.join(dataset_dir, "data.yaml")
with open(yaml_file_path, "w") as yaml_file:
    yaml_file.write(data_yaml_content.strip())
    
print("Created data.yaml:")
print(data_yaml_content)


Created data.yaml:

train: c:\Users\TDH\VIIIIIV\MVIIXXIV\2024\Assumption University 3y sem2\CSX_4201_Ai_Concepts\PROJECT\yolo_dataset\images\train
val: c:\Users\TDH\VIIIIIV\MVIIXXIV\2024\Assumption University 3y sem2\CSX_4201_Ai_Concepts\PROJECT\yolo_dataset\images\val
nc: 7
names: ['SellerName', 'SellerVAT', 'DocumentDate', 'ProductDescription', 'Quantity', 'Price', 'TotalDue']



In [2]:
import os
import json
import base64
import cv2
import glob

def yolo_to_labelme(image_path, yolo_label_path, classes):
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        return None
    height, width, _ = img.shape

    # Encode image in base64
    with open(image_path, "rb") as image_file:
        img_data = base64.b64encode(image_file.read()).decode("utf-8")

    shapes = []
    # Read YOLO annotation file
    with open(yolo_label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x_center, y_center, w, h = parts
            class_id = int(class_id)
            x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
            
            # Convert from normalized coordinates to pixel coordinates
            x_center *= width
            y_center *= height
            w *= width
            h *= height
            x_min = x_center - w / 2
            y_min = y_center - h / 2
            x_max = x_center + w / 2
            y_max = y_center + h / 2

            # Create a rectangle shape (LabelMe expects points for polygons)
            shape = {
                "label": classes[class_id] if class_id < len(classes) else str(class_id),
                "points": [[x_min, y_min], [x_max, y_max]],
                "group_id": None,
                "shape_type": "rectangle",
                "flags": {}
            }
            shapes.append(shape)

    # Build LabelMe JSON structure
    data = {
        "version": "4.5.6",  # LabelMe version (optional)
        "flags": {},
        "shapes": shapes,
        "imagePath": os.path.basename(image_path),
        "imageData": img_data,
        "imageHeight": height,
        "imageWidth": width
    }
    return data

def convert_dataset(yolo_images_dir, yolo_labels_dir, labelme_output_dir, classes):
    os.makedirs(labelme_output_dir, exist_ok=True)
    # Process each image file (assuming .jpg, adjust if needed)
    image_files = glob.glob(os.path.join(yolo_images_dir, "*.jpg"))
    for image_path in image_files:
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        yolo_label_path = os.path.join(yolo_labels_dir, base_name + ".txt")
        if not os.path.exists(yolo_label_path):
            print(f"Annotation not found for {image_path}")
            continue

        labelme_data = yolo_to_labelme(image_path, yolo_label_path, classes)
        if labelme_data is None:
            continue

        json_path = os.path.join(labelme_output_dir, base_name + ".json")
        with open(json_path, "w") as f:
            json.dump(labelme_data, f, indent=4)
        print(f"Saved {json_path}")

if __name__ == "__main__":
    # Define your classes (order should correspond to YOLO class ids)
    classes =  ['SellerName', 'SellerVAT', 'DocumentDate', 'ProductDescription', 'Quantity', 'Price', 'TotalDue']

    # Define the input folders for images and labels for each split.
    # For example, for training set:
    yolo_train_images = r"yolo_dataset\images\train"
    yolo_train_labels = r"yolo_dataset\labels\train"
    labelme_train_output = r"labelme_data"

    # Convert training set
    convert_dataset(yolo_train_images, yolo_train_labels, labelme_train_output, classes)

    # Similarly for validation set:
    yolo_val_images = r"yolo_dataset\images\val"
    yolo_val_labels = r"yolo_dataset\labels\val"
    labelme_val_output = r"labelme_data"

    convert_dataset(yolo_val_images, yolo_val_labels, labelme_val_output, classes)


Saved labelme_data\0.json
Saved labelme_data\1.json
Saved labelme_data\10.json
Saved labelme_data\100.json
Saved labelme_data\101.json
Saved labelme_data\102.json
Saved labelme_data\104.json
Saved labelme_data\105.json
Saved labelme_data\108.json
Saved labelme_data\109.json
Saved labelme_data\11.json
Saved labelme_data\110.json
Saved labelme_data\113.json
Saved labelme_data\116.json
Saved labelme_data\117.json
Saved labelme_data\118.json
Saved labelme_data\119.json
Saved labelme_data\121.json
Saved labelme_data\122.json
Saved labelme_data\123.json
Saved labelme_data\124.json
Saved labelme_data\126.json
Saved labelme_data\127.json
Saved labelme_data\128.json
Saved labelme_data\129.json
Saved labelme_data\13.json
Saved labelme_data\130.json
Saved labelme_data\131.json
Saved labelme_data\133.json
Saved labelme_data\134.json
Saved labelme_data\135.json
Saved labelme_data\136.json
Saved labelme_data\137.json
Saved labelme_data\138.json
Saved labelme_data\14.json
Saved labelme_data\140.json
